In [ ]:
def fff(x):
  with open(x) as file: 
    line = []
    for lines in file.readlines():
      line.append(lines)
    return line
line = fff('/content/drive/My Drive/Colab Notebooks/nlp_emotion/train.txt')

In [ ]:
line[0:5]

['i didnt feel humiliated;sadness\n',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake;sadness\n',
 'im grabbing a minute to post i feel greedy wrong;anger\n',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property;love\n',
 'i am feeling grouchy;anger\n']

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
def csv(line):
  list1,list2 = [],[]
  for lines in line:
    x,y = lines.split(';')
    y = y.replace('\n','')
    list1.append(x)
    list2.append(y)
  df = pd.DataFrame(list(list1),columns=['sentence'])
  df['emotion'] = list2
  return df

In [ ]:
df = csv(line)

In [ ]:
df

,sentence,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [ ]:
df.emotion.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: emotion, dtype: int64

In [ ]:
df.isnull().sum()

sentence    0
emotion     0
dtype: int64

##text preprocessing
####stopwords
####lematization
####count vectorizer


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
wn = WordNetLemmatizer()

In [ ]:
def lem(x):
  corpus = []
  i=1
  for words in x:
    words = words.split()
    y = [wn.lemmatize(word) for word in words if not word in stopwords.words('english')]
    y =  ' '.join(y)
    corpus.append(y)
  return corpus
x = lem(df['sentence'])

In [ ]:
x[:5]

['didnt feel humiliated',
 'go feeling hopeless damned hopeful around someone care awake',
 'im grabbing minute post feel greedy wrong',
 'ever feeling nostalgic fireplace know still property',
 'feeling grouchy']

##handling text data

In [ ]:
test_line = fff('/content/drive/My Drive/Colab Notebooks/nlp_emotion/test.txt') 

In [ ]:
test_df = csv(test_line)

In [ ]:
test_df[:5]

,sentence,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [ ]:
x_test = lem(test_df['sentence'])

In [ ]:
all = x + x_test

In [ ]:
len(all)

18000

labelling test and train data

In [ ]:
y = df.iloc[:,1].values

In [ ]:
y.shape

(16000,)

In [ ]:
y_test = test_df.iloc[:,1].values

In [ ]:
y_test.shape

(2000,)

###building and training the model

building an lstm model

In [ ]:
from tensorflow.keras.layers import Embedding,LSTM,Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [ ]:
y_train = pd.DataFrame(y)

In [ ]:
tokenizer = Tokenizer(nb_words=10000, split=' ')
tokenizer.fit_on_texts(all)
X1 = tokenizer.texts_to_sequences(all)
X1 = pad_sequences(X1,maxlen=20,padding='post',truncating='post')
Y1 = pd.get_dummies(y_train).values

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [ ]:
X_train = X1[:16000]
X_test = X1[16000:]

In [ ]:
Y_train = Y1

In [ ]:
Y_test = pd.get_dummies(y_test).values

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=10000,output_dim = 64,input_length=20))
model.add(LSTM(64))
model.add(Dense(6,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 64)            640000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 673,414
Trainable params: 673,414
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,batch_size=32,epochs=10,verbose=2,validation_split=0.2)

Epoch 1/10
400/400 - 2s - loss: 0.0099 - accuracy: 0.9618 - val_loss: 0.0280 - val_accuracy: 0.8978
Epoch 2/10
400/400 - 2s - loss: 0.0088 - accuracy: 0.9673 - val_loss: 0.0260 - val_accuracy: 0.8966
Epoch 3/10
400/400 - 2s - loss: 0.0083 - accuracy: 0.9686 - val_loss: 0.0265 - val_accuracy: 0.8944
Epoch 4/10
400/400 - 2s - loss: 0.0067 - accuracy: 0.9748 - val_loss: 0.0284 - val_accuracy: 0.8931
Epoch 5/10
400/400 - 2s - loss: 0.0065 - accuracy: 0.9754 - val_loss: 0.0263 - val_accuracy: 0.9000
Epoch 6/10
400/400 - 2s - loss: 0.0058 - accuracy: 0.9784 - val_loss: 0.0260 - val_accuracy: 0.9028
Epoch 7/10
400/400 - 2s - loss: 0.0053 - accuracy: 0.9786 - val_loss: 0.0252 - val_accuracy: 0.9025
Epoch 8/10
400/400 - 2s - loss: 0.0046 - accuracy: 0.9821 - val_loss: 0.0257 - val_accuracy: 0.9066
Epoch 9/10
400/400 - 2s - loss: 0.0046 - accuracy: 0.9834 - val_loss: 0.0276 - val_accuracy: 0.9006
Epoch 10/10
400/400 - 2s - loss: 0.0042 - accuracy: 0.9842 - val_loss: 0.0259 - val_accuracy: 0.9028

In [ ]:
loss,acc = model.evaluate(X_test,Y_test)

63/63 [==============================] - 0s 3ms/step - loss: 0.0284 - accuracy: 0.8970


In [ ]:
#model.save('/content/drive/My Drive/Colab Notebooks/nlp_emotion/model.h5')